In [39]:
import pandas as pd

In [40]:
# Set options to display more rows and columns
pd.set_option('display.max_rows', None)  # None to display all rows
pd.set_option('display.max_columns', None)  # None to display all columns


In [42]:
df = pd.read_csv("/Users/pratikhotchandani/Downloads/Github/This-week-in-football/reddit_submissions.csv")
df.head()

,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments
0,soccer,2024-09-29,1fse3w2,Link,/r/soccer/comments/1fse3w2/diego_simeone_and_h...,Diego Simeone and his players urging Atletico ...,6587,['**Mirrors / Alternative Angles**\n \n\n*I a...
1,soccer,2024-09-29,1fsewe3,Link,/r/soccer/comments/1fsewe3/atletico_madrid_1_1...,Atletico Madrid [1] - 1 Real Madrid - Angel Co...,2687,['**Mirrors / Alternative Angles**\n \n\n*I a...
2,soccer,2024-09-29,1fsfryn,Link,/r/soccer/comments/1fsfryn/atleticos_players_t...,Atletico's players thanking the Ultras after t...,2255,['**Mirrors / Alternative Angles**\n \n\n*I a...
3,soccer,2024-09-29,1fs7z6m,Link,/r/soccer/comments/1fs7z6m/bruno_fernandes_str...,Bruno Fernandes straight red card against Tott...,5829,['**Mirrors / Alternative Angles**\n \n\n*I a...
4,soccer,2024-09-29,1fs9viz,Link,/r/soccer/comments/1fs9viz/optajoe_23_manchest...,[OptaJoe] 23 - Manchester United have now lost...,4372,"[""**This is a stats thread. Remember that ther..."


In [6]:
# Calculate total number of characters in the comments column
total_characters = df['comments'].apply(lambda x: sum(len(comment) for comment in x)).sum()

print(f"Total number of characters in comments: {total_characters}")

Total number of characters in comments: 365491


In [33]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [34]:
%pip install langdetect

Note: you may need to restart the kernel to use updated packages.


In [35]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [36]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [4]:
from datetime import datetime, timedelta
from langdetect import detect
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer



In [5]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")


/Users/pratikhotchandani/opt/anaconda3/envs/torch/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/Users/pratikhotchandani/opt/anaconda3/envs/torch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
def translate_text(input_text, tgt_lang):
    # Step 1: Detect the source language
    src_lang = detect(input_text)

    # Step 2: Encode the input text
    encoded_input = tokenizer(input_text, return_tensors="pt", src_lang=src_lang)

    # Step 3: Generate the translated text
    translated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))

    # Step 4: Decode the translated text
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    return translated_text

In [18]:
text = comments.iloc[171].comments

In [20]:
import ast

# Convert the string to a list
sentences_list = ast.literal_eval(text)

# Display the list of sentences
print(sentences_list)

['Ich werde nie verstehen, wieso Hradecky in dem Spiel keine gelbe Karte bekommen hat', 'Find ich jetzt alles nicht schlimm. Bayer hat genau das getan was nötig war um zu Punkten. Bin überzeugt wenn sie anders als mit Terrorball gekommen wären wären sie übel versohlt worden. Einen guten coach macht halt auch pragmatismus in den richtigen Momenten aus, Hut ab vor Alonso dass er das richtig erkannt hatte. \n\nGehört zum Spiel. Und ich freue mich eher über Kompanyball und was er bisher geschafft hat aus unserem Fußballspiel herauszuholen (vergleich mal mit Tuchel lol) als mich über Leverkusen aufzuregen.', 'Ich respektiere die Ehrlichkeit, aber dass Leverkusen das offenbar für nötig hielt, sagt einiges über die Kräfteverhältnisse in diesem Spiel aus.', 'Bin biased und ja ich weiß, das ist "normal" (Effe gerade im Dopa), aber ich find\'s billig, unsportlich und arm.', 'tja würde bei jeder Unterbrechung einfach die Zeit gestoppt, müssten wir über solche Dinge gar nicht diskutieren.\n"AbEr d

In [22]:

def translate_text(input_text, tgt_lang):
    # Step 1: Detect the source language
    src_lang = detect(input_text)

    # Step 2: Set the source language for the tokenizer
    tokenizer.src_lang = src_lang

    # Step 3: Encode the input text
    encoded_input = tokenizer(input_text, return_tensors="pt")

    # Step 4: Generate the translated text
    generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))

    # Step 5: Decode the translated text
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return src_lang, translated_text

# Define the target language
target_language = 'en'  # Change this to your desired target language

# Prepare a list to hold the results
results = []

# Iterate over the sentences and translate each one
for text in sentences_list:
    detected_lang, translated_text = translate_text(text, target_language)
    results.append({
        'original_text': text,
        'detected_language': detected_lang,
        'final_translated_text': translated_text
    })

# Create a DataFrame from the results
translations_df = pd.DataFrame(results)

# Display the resulting DataFrame
print(translations_df)

                                       original_text detected_language  \
0  Ich werde nie verstehen, wieso Hradecky in dem...                de   
1  Find ich jetzt alles nicht schlimm. Bayer hat ...                de   
2  Ich respektiere die Ehrlichkeit, aber dass Lev...                de   
3  Bin biased und ja ich weiß, das ist "normal" (...                de   
4  tja würde bei jeder Unterbrechung einfach die ...                de   
5  Leverkusen hat gestern gefühlt ab der 1. Minut...                de   
6  Imagine du gibst so etwas zu und bist trotzdem...                de   
7             Leverkusen literarisch deutsche Getafe                de   
8  Was im Profifußball wird auf FairPlay geschiss...                de   
9  Zeitspiel beim Gegner nervt immer. Bayern hatt...                de   

                               final_translated_text  
0  I will never understand why Hradecky did not g...  
1  I find everything not bad now. Bayer has done ...  
2  I respect the hon

In [25]:
translations_df.to_csv("translations.csv")

In [43]:
# Convert the string to a list
import ast
df['comments'] = df['comments'].apply(ast.literal_eval)

In [44]:
# Explode the 'comments' column
df_exploded = df.explode('comments')

In [46]:
df_exploded = df_exploded.reset_index(drop=True)


In [47]:
df_exploded.sample(15)

,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments
1616,Bundesliga,2024-09-26,1fq6xpe,Link,/r/Bundesliga/comments/1fq6xpe/europa_league_e...,Europa League: Eintracht Frankfurt gibt Zwei-T...,69,"Am Ende patzt halt Santos, aber dem kannste au..."
1740,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,Neuer Machtkampf in Köln: Mitglieder verweiger...,21,Da der Kicker-Artikel nicht viel an Hintergrün...
1333,Bundesliga,2024-09-29,1fs0j0y,Link,/r/Bundesliga/comments/1fs0j0y/vfb_legt_einspr...,VFB legt Einspruch gegen Gelb-Rote-Karte ein,134,"VfB. Kleines ""F""."
1312,Bundesliga,2024-09-29,1fs6xaf,Link,/r/Bundesliga/comments/1fs6xaf/auch_kiels_mach...,Auch Kiels Machino trifft doppelt: Marmoush-Sh...,69,Nächste Woche machen wir die Meisterschaft geg...
646,seriea,2024-09-23,1fnqq5w,Image,/r/seriea/comments/1fnqq5w/capture_the_flag/,Capture the flag,425,One was for the scudetto the other is the firs...
1052,PremierLeague,2024-09-24,1foik48,Link,/r/PremierLeague/comments/1foik48/thierry_henr...,Thierry Henry on the crowded schedule discours...,2051,"It is what it is now. Truly, the matches are t..."
1739,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,Neuer Machtkampf in Köln: Mitglieder verweiger...,21,Am besten jemand kompetentes übernimmt. Beispi...
7,soccer,2024-09-29,1fse3w2,Link,/r/soccer/comments/1fse3w2/diego_simeone_and_h...,Diego Simeone and his players urging Atletico ...,6587,Both Real Madrid and Barcelona have banned the...
1608,Bundesliga,2024-09-27,1fqm6ox,Text,/r/Bundesliga/comments/1fqm6ox/wo_finde_ich_gu...,Wo finde ich gute Bundesliga Statistiken?,13,fbref.com
111,soccer,2024-09-29,1fse1he,Link,/r/soccer/comments/1fse1he/simeone_and_koke_go...,Simeone and Koke go over to try and ease the h...,1217,Then they throw shit at gimenez lmao


In [48]:
df_exploded.to_csv("exploded_dataset.csv")

In [25]:
df = df[df['subreddit'] == 'Bundesliga']
df

,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments
168,Bundesliga,2024-09-29,1fs9r1o,Link,/r/Bundesliga/comments/1fs9r1o/43_nach_03_brem...,4:3 nach 0:3: Bremen zerlegt Hoffenheim nach A...,244,[Ein gewohnt geruhsamer Bremer Bundesligaauftr...
169,Bundesliga,2024-09-29,1fs03er,Link,/r/Bundesliga/comments/1fs03er/904_konstantino...,90+4' Konstantinos Koulierakis [VfL Wolfsburg]...,511,[Sowas ist einfach nur unsportlich und sollte ...
170,Bundesliga,2024-09-29,1fs3gf3,Link,/r/Bundesliga/comments/1fs3gf3/vereine_der_pre...,"Vereine der Premier League planen laut ""Times""...",189,[Ein guter Vorschlag aus der Premier League? D...
171,Bundesliga,2024-09-29,1fs0yyr,Link,/r/Bundesliga/comments/1fs0yyr/bayers_xhaka_gi...,"Bayers Xhaka gibt Zeitspiel zu: ""Wir mussten e...",244,"[Ich werde nie verstehen, wieso Hradecky in de..."
172,Bundesliga,2024-09-29,1fs6xaf,Link,/r/Bundesliga/comments/1fs6xaf/auch_kiels_mach...,Auch Kiels Machino trifft doppelt: Marmoush-Sh...,69,[Nächste Woche machen wir die Meisterschaft ge...
173,Bundesliga,2024-09-29,1fsfk86,Text,/r/Bundesliga/comments/1fsfk86/3_liga_08_spiel...,"[3. Liga | 08. Spieltag] Alle Ergebnisse, Spie...",15,"[i dont feel so good, 8 Spieltage durch, und i..."
174,Bundesliga,2024-09-29,1fs0j0y,Link,/r/Bundesliga/comments/1fs0j0y/vfb_legt_einspr...,VFB legt Einspruch gegen Gelb-Rote-Karte ein,134,"[Wird, wie bei Schallenberg gegen Nürnberg, wo..."
175,Bundesliga,2024-09-29,1fs296f,Link,/r/Bundesliga/comments/1fs296f/arnold_am_prang...,Arnold am Pranger - So erklärt er die Karazor-...,86,[Das er eigentlich nicht nennenswert getroffen...
176,Bundesliga,2024-09-29,1fs4tth,Text,/r/Bundesliga/comments/1fs4tth/fußballspektake...,Fußball-Spektakel zwischen Köln und dem KSC: E...,55,"[Völlig normales Zweitligaspiel für mich, ein ..."
177,Bundesliga,2024-09-29,1fs4crd,Link,/r/Bundesliga/comments/1fs4crd/klarer_auswärts...,Klarer Auswärtssieg: Elversberg schockt die He...,35,"[Ich spüre Dardais Comeback schon , Hab das Ge..."


In [6]:
def translate_text(input_text, tgt_lang, tokenizer, model):
    # Step 1: Detect the source language
    src_lang = detect(input_text)

    if src_lang != 'en':

        # Step 2: Set the source language for the tokenizer
        tokenizer.src_lang = src_lang

        # Step 3: Encode the input text
        encoded_input = tokenizer(input_text, return_tensors="pt")

        # Step 4: Generate the translated text
        generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.get_lang_id(tgt_lang))

        # Step 5: Decode the translated text
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

        return src_lang, translated_text
    
    else:
        return src_lang, "translation not required"

In [29]:
# Function to apply the translation to each comment in the DataFrame
def translate_comments(df, column_name, tgt_lang, tokenizer, model):
    # Initialize new columns for detected language and dynamic translated text
    translated_column_name = f"{column_name}_translated_text"  # Create dynamic column name
    language_column_name = f"{column_name}_langugage"  # Create dynamic column name
    df[language_column_name] = None
    df[translated_column_name] = None

    # Iterate over each row and apply the translation
    for idx in df.index:
        comment = df.at[idx, column_name]  # Access the comment in the given row
        try:
            # Translate the comment
            detected_lang, translated_comment = translate_text(comment, tgt_lang, tokenizer, model)

            # If translation is not required, keep the original comment
            if translated_comment == "translation not required":
                df.at[idx, translated_column_name] = comment
            else:
                df.at[idx, translated_column_name] = translated_comment

            df.at[idx, language_column_name] = detected_lang

        except Exception as e:
            print(f"Error translating comment at index {idx}: {comment}, error: {e}")
            df.at[idx, language_column_name] = None
            df.at[idx, translated_column_name] = None

    return df


In [30]:
import ast

def prepare_data_for_translation(df, column_name):
    df[column_name] = df[column_name].apply(ast.literal_eval)
    df_exploded = df.explode(column_name)
    df_exploded = df_exploded.reset_index(drop=True)
    return df_exploded


In [31]:
df = df[df['subreddit'] == 'Bundesliga'].sample(10)
df = prepare_data_for_translation(df, 'comments')
#translated_data = translate_comments(df, 'comments', 'en', tokenizer, model)


SyntaxError: invalid syntax (<unknown>, line 1)

In [27]:
translated_data.head()

,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments,detected_language,translated_text
0,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Bedeutet die Verweigerung der Entlastung einfa...,de,Does the rejection of the dismissal simply mea...
1,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Am besten jemand kompetentes übernimmt. Beispi...,de,The best one is to take over competent. for ex...
2,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Da der Kicker-Artikel nicht viel an Hintergrün...,de,Since the Kicker article does not provide a lo...
3,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Harter Tobak. War meines Wissens das letzte ma...,de,Harter Tobak. I know it was the last time at t...
4,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Der Medienzirkus freut sich mal wieder. Endlic...,de,The media circus rejoiced once again. Finally ...


In [11]:
translated_data.to_csv("bundesliga.csv")

In [18]:
## Translating title

In [28]:
translated_data.head()

,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments,detected_language,translated_text
0,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Bedeutet die Verweigerung der Entlastung einfa...,de,Does the rejection of the dismissal simply mea...
1,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Am besten jemand kompetentes übernimmt. Beispi...,de,The best one is to take over competent. for ex...
2,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Da der Kicker-Artikel nicht viel an Hintergrün...,de,Since the Kicker article does not provide a lo...
3,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Harter Tobak. War meines Wissens das letzte ma...,de,Harter Tobak. I know it was the last time at t...
4,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Der Medienzirkus freut sich mal wieder. Endlic...,de,The media circus rejoiced once again. Finally ...


In [22]:
# Create a new DataFrame where each submission_id is unique
unique_submission_df = translated_data.drop_duplicates(subset='submission_id', keep='first')

,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments,detected_language,translated_text
0,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,Neuer Machtkampf in Köln: Mitglieder verweiger...,21,Bedeutet die Verweigerung der Entlastung einfa...,de,Does the rejection of the dismissal simply mea...
5,Bundesliga,2024-09-27,1fqmw0f,Link,/r/Bundesliga/comments/1fqmw0f/schneller_und_s...,Schneller und stressfreier Einlass ins Stadion...,87,![gif](giphy|IeKgCDlpTqRQbZEhBF),en,![gif](giphy|IeKgCDlpTqRQbZEhBF)
15,Bundesliga,2024-09-24,1fohw4h,Text,/r/Bundesliga/comments/1fohw4h/hall_of_shame_s...,Hall of Shame Statistiken eurer Vereine?,66,"Nicht mein Verein, aber der meines Freundes, d...",de,"Not my club, but my friend who always filled m..."
25,Bundesliga,2024-09-28,1frjf6v,Text,/r/Bundesliga/comments/1frjf6v/jablonski_gibt_...,Jablonski gibt Fehlentscheidung zu und kritisi...,140,Eigentlich müsste Arnold gesperrt werden.\n\nW...,de,"In fact, Arnold must be locked. How wicked he ..."
35,Bundesliga,2024-09-25,1fp9hpz,Text,/r/Bundesliga/comments/1fp9hpz/fc_midtjylland_...,FC Midtjylland - TSG Hoffenheim 1899 | UEFA Eu...,0,"Defensiv absolut gruselig, ohne Baumann wären ...",de,"Defensive absolutely terrible, without Baumann..."
41,Bundesliga,2024-09-27,1fqrlpo,Link,/r/Bundesliga/comments/1fqrlpo/planungssicherh...,Planungssicherheit zwischen den blau-weißen Pf...,14,"Immer noch sau schade, dass wir ihn nicht halt...",de,It’s a sad thing that we couldn’t keep him.
46,Bundesliga,2024-09-25,1fp788h,Text,/r/Bundesliga/comments/1fp788h/verbandspokale_...,Verbandspokale 2024/25 - die ersten Runden,51,Ganz viel Liebe für diese Zusammenfassung <3,de,A lot of love for this summary <3
56,Bundesliga,2024-09-25,1fp72ge,Link,/r/Bundesliga/comments/1fp72ge/fc_bayern_steht...,FC Bayern steht kurz vor der Verpflichtung von...,34,"Keine Ahnung, ob der was kann. In seiner Zeit ...",de,In his time as a sports director at Borussia h...
66,Bundesliga,2024-09-28,1frpvyt,Link,/r/Bundesliga/comments/1frpvyt/ricken_widerspr...,Ricken widerspricht Streit-Berichten - und läs...,4,"Naja die befinden sich in Verhandlungen, das k...",de,"Well the ones are in negotiations, that sounds..."
67,Bundesliga,2024-09-26,1fq6xpe,Link,/r/Bundesliga/comments/1fq6xpe/europa_league_e...,Europa League: Eintracht Frankfurt gibt Zwei-T...,69,Ich hab noch nie so eine dumme letzte halbe St...,de,I’ve never seen such a foolish last half an ho...


In [23]:
translated_data_title = translate_comments(unique_submission_df, 'submission_title', 'en', tokenizer, model)

/var/folders/dq/pg0mssxn6yzdwtcdrwnt0bs80000gn/T/ipykernel_69360/3866694205.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['detected_language'] = None
/var/folders/dq/pg0mssxn6yzdwtcdrwnt0bs80000gn/T/ipykernel_69360/3866694205.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[translated_column_name] = None


In [24]:
translated_data_title

,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments,detected_language,translated_text,submission_title_translated_text
0,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,Neuer Machtkampf in Köln: Mitglieder verweiger...,21,Bedeutet die Verweigerung der Entlastung einfa...,de,Does the rejection of the dismissal simply mea...,New power struggle in Cologne: Members refuse ...
5,Bundesliga,2024-09-27,1fqmw0f,Link,/r/Bundesliga/comments/1fqmw0f/schneller_und_s...,Schneller und stressfreier Einlass ins Stadion...,87,![gif](giphy|IeKgCDlpTqRQbZEhBF),de,![gif](giphy|IeKgCDlpTqRQbZEhBF),Fast and stress-free entry to the stadium: FC ...
15,Bundesliga,2024-09-24,1fohw4h,Text,/r/Bundesliga/comments/1fohw4h/hall_of_shame_s...,Hall of Shame Statistiken eurer Vereine?,66,"Nicht mein Verein, aber der meines Freundes, d...",de,"Not my club, but my friend who always filled m...",Hall of Shame statistics of your associations?
25,Bundesliga,2024-09-28,1frjf6v,Text,/r/Bundesliga/comments/1frjf6v/jablonski_gibt_...,Jablonski gibt Fehlentscheidung zu und kritisi...,140,Eigentlich müsste Arnold gesperrt werden.\n\nW...,de,"In fact, Arnold must be locked. How wicked he ...",Jablonski acknowledges mistake and criticizes ...
35,Bundesliga,2024-09-25,1fp9hpz,Text,/r/Bundesliga/comments/1fp9hpz/fc_midtjylland_...,FC Midtjylland - TSG Hoffenheim 1899 | UEFA Eu...,0,"Defensiv absolut gruselig, ohne Baumann wären ...",en,"Defensive absolutely terrible, without Baumann...",FC Midtjylland - TSG Hoffenheim 1899 | UEFA Eu...
41,Bundesliga,2024-09-27,1fqrlpo,Link,/r/Bundesliga/comments/1fqrlpo/planungssicherh...,Planungssicherheit zwischen den blau-weißen Pf...,14,"Immer noch sau schade, dass wir ihn nicht halt...",de,It’s a sad thing that we couldn’t keep him.,Planning security between the blue-white poste...
46,Bundesliga,2024-09-25,1fp788h,Text,/r/Bundesliga/comments/1fp788h/verbandspokale_...,Verbandspokale 2024/25 - die ersten Runden,51,Ganz viel Liebe für diese Zusammenfassung <3,de,A lot of love for this summary <3,2024/25 – the first round
56,Bundesliga,2024-09-25,1fp72ge,Link,/r/Bundesliga/comments/1fp72ge/fc_bayern_steht...,FC Bayern steht kurz vor der Verpflichtung von...,34,"Keine Ahnung, ob der was kann. In seiner Zeit ...",de,In his time as a sports director at Borussia h...,FC Bayern is shortly ahead of the commitment o...
66,Bundesliga,2024-09-28,1frpvyt,Link,/r/Bundesliga/comments/1frpvyt/ricken_widerspr...,Ricken widerspricht Streit-Berichten - und läs...,4,"Naja die befinden sich in Verhandlungen, das k...",de,"Well the ones are in negotiations, that sounds...",Ricken opposes disputed reports - and leaves K...
67,Bundesliga,2024-09-26,1fq6xpe,Link,/r/Bundesliga/comments/1fq6xpe/europa_league_e...,Europa League: Eintracht Frankfurt gibt Zwei-T...,69,Ich hab noch nie so eine dumme letzte halbe St...,de,I’ve never seen such a foolish last half an ho...,Europa League: Eintracht Frankfurt gives two-d...


In [25]:
# Ensure both dataframes have the required columns
df2_reduced = translated_data_title[['submission_id', 'submission_title_translated_text']]

# Merge df1 with df2 on submission_id
merged_df = translated_data.merge(df2_reduced, on='submission_id', how='left')

# Replace the values in submission_title column in df1 with the translated title from df2
translated_data['submission_title'] = merged_df['submission_title_translated_text'].combine_first(translated_data['submission_title'])

translated_data


,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments,detected_language,translated_text
0,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Bedeutet die Verweigerung der Entlastung einfa...,de,Does the rejection of the dismissal simply mea...
1,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Am besten jemand kompetentes übernimmt. Beispi...,de,The best one is to take over competent. for ex...
2,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Da der Kicker-Artikel nicht viel an Hintergrün...,de,Since the Kicker article does not provide a lo...
3,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Harter Tobak. War meines Wissens das letzte ma...,de,Harter Tobak. I know it was the last time at t...
4,Bundesliga,2024-09-25,1foyovw,Link,/r/Bundesliga/comments/1foyovw/neuer_machtkamp...,New power struggle in Cologne: Members refuse ...,21,Der Medienzirkus freut sich mal wieder. Endlic...,de,The media circus rejoiced once again. Finally ...
5,Bundesliga,2024-09-27,1fqmw0f,Link,/r/Bundesliga/comments/1fqmw0f/schneller_und_s...,Fast and stress-free entry to the stadium: FC ...,87,![gif](giphy|IeKgCDlpTqRQbZEhBF),en,![gif](giphy|IeKgCDlpTqRQbZEhBF)
6,Bundesliga,2024-09-27,1fqmw0f,Link,/r/Bundesliga/comments/1fqmw0f/schneller_und_s...,Fast and stress-free entry to the stadium: FC ...,87,Ach deshalb hat mich keiner gegen Zagreb abget...,de,"Therefore, no one has struck me against Zagreb."
7,Bundesliga,2024-09-27,1fqmw0f,Link,/r/Bundesliga/comments/1fqmw0f/schneller_und_s...,Fast and stress-free entry to the stadium: FC ...,87,Wie kann das Sicherheitskonzept denn bitte abg...,de,How can the security concept be removed? as if...
8,Bundesliga,2024-09-27,1fqmw0f,Link,/r/Bundesliga/comments/1fqmw0f/schneller_und_s...,Fast and stress-free entry to the stadium: FC ...,87,In den USA ist das seit ewigkeiten Gang und Gä...,de,In the U.S. it has been for ever to go through...
9,Bundesliga,2024-09-27,1fqmw0f,Link,/r/Bundesliga/comments/1fqmw0f/schneller_und_s...,Fast and stress-free entry to the stadium: FC ...,87,Wenn es wirklich so wie beschrieben funktionie...,de,If it really works as described and only climb...


In [50]:
df_exp = pd.read_csv("exploded_dataset.csv", na_filter=False)


In [55]:
df_exp[df_exp['Category'] != ""]

,Unnamed: 0,subreddit,submission_date,submission_id,submission_type,submission_url,submission_title,no_of_upvotes,comments,Category
0,36,soccer,9/29/24,1fs7z6m,Link,/r/soccer/comments/1fs7z6m/bruno_fernandes_str...,Bruno Fernandes straight red card against Tott...,5829,[deleted],NaN
1,215,soccer,9/29/24,1fryycx,Link,/r/soccer/comments/1fryycx/crazy_goal_line_cle...,Crazy Goal Line Clearance by Kawase in Hong Ko...,8013,Waaaa!,NaN
2,452,football,9/24/24,1fo9vsq,Link,/r/football/comments/1fo9vsq/arteta_on_arsenal...,Arteta on Arsenal's approach after going down ...,402,Football,NaN
3,511,football,9/23/24,1fnoem0,Image,/r/football/comments/1fnoem0/interesting_stat_...,Interesting stat on time wasting from goal kicks,19,[deleted],NaN
4,513,football,9/23/24,1fnkaeg,Link,/r/football/comments/1fnkaeg/friedkin_group_ag...,Friedkin Group agrees Everton takeover deal,2,No comments,NaN
5,1,soccer,9/29/24,1fse3w2,Link,/r/soccer/comments/1fse3w2/diego_simeone_and_h...,Diego Simeone and his players urging Atletico ...,6587,That's not even smart. Why are they doing this?,Human-Conversation
6,2,soccer,9/29/24,1fse3w2,Link,/r/soccer/comments/1fse3w2/diego_simeone_and_h...,Diego Simeone and his players urging Atletico ...,6587,Incapable of being decent humans. Your own man...,Human-Conversation
7,4,soccer,9/29/24,1fse3w2,Link,/r/soccer/comments/1fse3w2/diego_simeone_and_h...,Diego Simeone and his players urging Atletico ...,6587,I like Atleti and while you can claim this is ...,Human-Conversation
8,43,soccer,9/29/24,1fs9viz,Link,/r/soccer/comments/1fs9viz/optajoe_23_manchest...,[OptaJoe] 23 - Manchester United have now lost...,4372,>> Lose few games \n\n>> Get back to back draw...,Human-Conversation
9,48,soccer,9/29/24,1fs9viz,Link,/r/soccer/comments/1fs9viz/optajoe_23_manchest...,[OptaJoe] 23 - Manchester United have now lost...,4372,I'll copy and paste my stat from the DD:\n\n10...,Human-Conversation
